## Test of flux limiters for finite-volume (conservative) LW scheme
This is a one-dimensional simulation of Burger's equation using conservative schemes in the range $0 < x < 100$.

$$
\frac{\partial u}{\partial t} = c \frac{\partial u}{\partial x} 
$$


In [1]:
import numpy as np
import limiters as lim

In [2]:
#%%time
# Grid
nx = 100
xl, xr = 0., 100.
nghost = 2

dx = (xr - xl) / nx
nxt = nx + 2 * nghost

# Courant number
nu = 0.8

# Advection speed (constant)
c = 1.

# Scheme
# 'lw'
scheme = 'lw'

# Output dir
out_dir = 'c-lax-wendroff'

# limiter
# 'minmod', '' for no limiter

limiter = 'superbee'

# Constant timestep
dt = nu * dx / c

# End time
t_end = 30.

# I/O
n_output = 1

# Initialize
t = 0.
nstep = 0

In [3]:
# Initialize grid

# Cell centered grid
xc = np.linspace(xl-0.5*nghost*dx, xr+0.5*nghost*dx, nxt)

In [4]:
# Get Limiter function
flim = getattr(lim, limiter) if limiter else None

In [5]:
# Initial conditions (Top hat)
#u = np.where(np.logical_and(np.greater(xc, 25), np.less(xc, 50)), 1., 0.)
u = np.where(xc<50., 1., 0.)

In [6]:
#%%time
# Time integration loop
while t < t_end:
    
    # Set Boundary conditions
    u[:nghost] = u[nghost+1]
    u[-nghost:] = u[-nghost-1]
    
    # Create left, right, and inner arrays to construct fluxes and limiters
    
    # Cell center fluxes
    u_left = u[nghost-1:-nghost]
    u_right = u[nghost:-nghost+1]
    fc_left = c * u_left
    fc_right = c * u_right

    # Right and 
    if scheme == 'lw':
        
        # Limiters for left and right fluxes
        if flim:
            u_inner = u[nghost-1:-nghost+1]
            u_left = u[nghost-2:-nghost]
            u_right = u[nghost:-nghost+2] if nghost > 2 else u[nghost:]
            flim_n = flim(u_left, u_inner, u_right)
            flim_l = flim_n[:-2]
            flim_r = flim_n[1:-1]
                
        else:
            flim_r = flim_l = 1.

        # Low res flux
        fn_low = fc_left
        fn_low_right = fn_low[1:]
        fn_low_left = fn_low[:-1]
        
        # High res flux
        fn_high = 0.5 * (1. - nu) * (fc_right - fc_left)
        fn_high_right = fn_high[1:] * flim_r
        fn_high_left = fn_high[:-1] * flim_l

        # Left and right fluxes are fn_low + fn_high
        fn_right = fn_low_right + fn_high_right
        fn_left = fn_low_left + fn_high_left

    else:
        print('Error: Unknown scheme. Exiting.')
        exit()

    # Calculate new solution. One timestep in selected scheme.
    u_inner = u[nghost:-nghost]
    u_inner = u_inner - nu * (fn_right - fn_left)
    
    # Extend u to include ghost cells again
    u = np.concatenate((nghost*[0], u_inner, nghost*[0]))
    
    # Update time
    t = t + dt
    nstep = nstep + 1
    
    # I/O
    if nstep % n_output == 0:
        u_inner.tofile(f'{out_dir}/{scheme}-{int(nstep/n_output):0>4d}.dat')
        